In [1]:
!python -m spacy download en_core_web_sm


C:\Users\gthum\anaconda3\python.exe: No module named spacy


In [2]:
import os
print(os.path.isfile("symptom_disease_model.onnx"))


False


In [3]:
import pandas as pd

# Sample synthetic data: Symptoms and their associated disease
data = {
    "fever": [1, 0, 1, 0, 1, 0],
    "cough": [1, 1, 0, 0, 1, 0],
    "fatigue": [1, 1, 1, 0, 0, 0],
    "headache": [0, 1, 0, 1, 0, 1],
    "nausea": [0, 0, 1, 1, 0, 1],
    "disease": ["Flu", "Cold", "COVID", "Migraine", "Flu", "Food Poisoning"]
}

# Create DataFrame and save it
df = pd.DataFrame(data)
df.to_csv("symptom_disease_data.csv", index=False)
print("Sample dataset created!")
df.head()


Sample dataset created!


,fever,cough,fatigue,headache,nausea,disease
0,1,1,1,0,0,Flu
1,0,1,1,1,0,Cold
2,1,0,1,0,1,COVID
3,0,0,0,1,1,Migraine
4,1,1,0,0,0,Flu


In [4]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# Load dataset
df = pd.read_csv("symptom_disease_data.csv")

# Separate features and target
X = df.drop(columns=["disease"])
y = df["disease"]

# Encode target labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.2, random_state=42)


In [5]:
from sklearn.ensemble import RandomForestClassifier

# Train a Random Forest model
clf = RandomForestClassifier(n_estimators=100, random_state=42)
clf.fit(X_train, y_train)

# Test accuracy
accuracy = clf.score(X_test, y_test)
print(f"Model accuracy: {accuracy * 100:.2f}%")


Model accuracy: 50.00%


In [6]:
!pip install onnx skl2onnx


  Obtaining dependency information for onnx from https://files.pythonhosted.org/packages/51/a5/19b0dfcb567b62e7adf1a21b08b23224f0c2d13842aee4d0abc6f07f9cf5/onnx-1.17.0-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for skl2onnx from https://files.pythonhosted.org/packages/08/de/e8825727acd80484aa28080de62e4dc21f076d6887c10db49e2c8a66578f/skl2onnx-1.18.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for onnxconverter-common>=1.7.0 from https://files.pythonhosted.org/packages/6d/6a/9ed9fd4da34cb41fda35bc5cc9e990c605689db7de63ed84fedbca5a77f6/onnxconverter_common-1.14.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for protobuf>=3.20.2 from https://files.pythonhosted.org/packages/8b/e6/2a47ce2eba1aaf287380a44270da897ada03d118a55c19595ec7b4f0831f/protobuf-3.20.2-py2.py3-none-any.whl.metadata
   ---------------------------------------- 0.0/14.5 MB ? eta -:--:--
   ---------------------------------------- 0.0/14.5 MB 330.3 kB/s eta

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
grpcio-status 1.68.0 requires protobuf<6.0dev,>=5.26.1, but you have protobuf 3.20.2 which is incompatible.


In [7]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

In [8]:


# Convert model to ONNX
initial_type = [("float_input", FloatTensorType([None, X_train.shape[1]]))]
onnx_model = convert_sklearn(clf, initial_types=initial_type)

# Save the ONNX model
with open("symptom_disease_model.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

print("ONNX model saved successfully!")


ONNX model saved successfully!


In [10]:
!pip install onnxruntime

import onnxruntime as ort
import numpy as np
import json

# Load the ONNX model
model_path = "symptom_disease_model.onnx"
session = ort.InferenceSession(model_path)

# Symptom list (based on training data)
symptom_list = ["fever", "cough", "fatigue", "headache", "nausea"]

# Disease labels (make sure it matches the label encoder’s order)
disease_labels = ["Cold", "COVID", "Flu", "Food Poisoning", "Migraine"]

print("Model and symptom data loaded successfully!")


  Obtaining dependency information for onnxruntime from https://files.pythonhosted.org/packages/2c/2d/2d4d202c0bcfb3a4cc2b171abb9328672d7f91d7af9ea52572722c6d8d96/onnxruntime-1.20.1-cp311-cp311-win_amd64.whl.metadata
  Obtaining dependency information for coloredlogs from https://files.pythonhosted.org/packages/a7/06/3d6badcf13db419e25b07041d9c7b4a2c331d3f4e7134445ec5df57714cd/coloredlogs-15.0.1-py2.py3-none-any.whl.metadata
  Obtaining dependency information for flatbuffers from https://files.pythonhosted.org/packages/b8/25/155f9f080d5e4bc0082edfda032ea2bc2b8fab3f4d25d46c1e9dd22a1a89/flatbuffers-25.2.10-py2.py3-none-any.whl.metadata
  Obtaining dependency information for humanfriendly>=9.1 from https://files.pythonhosted.org/packages/f0/0f/310fb31e39e2d734ccaa2c0fb981ee41f7bd5056ce9bc29b2248bd569169/humanfriendly-10.0-py2.py3-none-any.whl.metadata
  Obtaining dependency information for pyreadline3 from https://files.pythonhosted.org/packages/5a/dc/491b7661614ab97483abf2056be1deee4dc24

In [12]:
!pip install SpeechRecognition

import speech_recognition as sr

# Speech recognition function
def recognize_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("Listening for symptoms...")
        audio = recognizer.listen(source)
    try:
        text = recognizer.recognize_google(audio)
        print(f"Recognized Symptoms: {text}")
        return text.lower()
    except sr.UnknownValueError:
        print("Could not understand audio")
        return ""
    except sr.RequestError:
        print("Speech recognition service unavailable")
        return ""


  Obtaining dependency information for SpeechRecognition from https://files.pythonhosted.org/packages/09/47/5dcfcd8a2c8c2981986fc196e98fc57bc1ecb5233b2d54dac0c0d448b019/SpeechRecognition-3.14.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/32.9 MB ? eta -:--:--
   ---------------------------------------- 0.1/32.9 MB 1.5 MB/s eta 0:00:22
   ---------------------------------------- 0.3/32.9 MB 3.9 MB/s eta 0:00:09
   -- ------------------------------------- 1.8/32.9 MB 12.6 MB/s eta 0:00:03
   ---- ----------------------------------- 3.8/32.9 MB 20.5 MB/s eta 0:00:02
   ------ --------------------------------- 5.4/32.9 MB 24.6 MB/s eta 0:00:02
   ------ --------------------------------- 5.6/32.9 MB 23.7 MB/s eta 0:00:02
   --------- ------------------------------ 7.9/32.9 MB 24.1 MB/s eta 0:00:02
   ------------ --------------------------- 9.9/32.9 MB 26.3 MB/s eta 0:00:01
   -------------- ------------------------- 12.0/32.9 MB 36.4 MB/s eta 0:00:01
   ------

In [13]:
# Convert recognized symptoms into model input
def symptoms_to_vector(symptoms):
    input_vector = np.zeros(len(symptom_list))
    for symptom in symptom_list:
        if symptom in symptoms:
            index = symptom_list.index(symptom)
            input_vector[index] = 1
    return input_vector.reshape(1, -1).astype(np.float32)

# Predict disease using ONNX model
def predict_disease(symptoms):
    input_vector = symptoms_to_vector(symptoms)
    output = session.run(None, {session.get_inputs()[0].name: input_vector})
    # output = session.run(None, {session.get_inputs()[0].name: input_vector})
    print(f"Model Output: {output}")
    print(f"Output Type: {type(output)}")
    #print(f"Output Shape: {[o.shape for o in output]}")

    # predicted_label = np.argmax(output[0])
    # predicted_disease = disease_labels[predicted_label]
    # print(f"Predicted Disease: {predicted_disease}")

    predicted_label = output[0][0]  # Directly taking the class index from the array
    predicted_disease = disease_labels[predicted_label]  # Assuming disease_labels is your mapping
    print(f"Predicted Disease: {predicted_disease}")

    return predicted_disease


In [15]:
!pip install pyaudio


  Obtaining dependency information for pyaudio from https://files.pythonhosted.org/packages/82/d8/f043c854aad450a76e476b0cf9cda1956419e1dacf1062eb9df3c0055abe/PyAudio-0.2.14-cp311-cp311-win_amd64.whl.metadata
   ---------------------------------------- 0.0/164.1 kB ? eta -:--:--
   -- ------------------------------------- 10.2/164.1 kB ? eta -:--:--
   ---- ---------------------------------- 20.5/164.1 kB 165.2 kB/s eta 0:00:01
   --------- ----------------------------- 41.0/164.1 kB 281.8 kB/s eta 0:00:01
   -------------------------- ----------- 112.6/164.1 kB 656.4 kB/s eta 0:00:01
   -------------------------------------- 164.1/164.1 kB 822.6 kB/s eta 0:00:00


In [17]:
def main():
    symptoms = recognize_speech()
    if symptoms:
        predict_disease(symptoms)

if __name__ == "__main__":
    main()


Listening for symptoms...
Could not understand audio
